In this notebook we will explore how to define the concept "churn" and how we will predict the churn rate that we define.

In [2]:
import configparser 
from datetime import datetime 
import os 
from pyspark.sql import SparkSession 
from pyspark.sql.functions import udf, col 
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
config = configparser.ConfigParser() 

In [4]:
os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key_id 
os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_access_key
os.environ['AWS_REGION_NAME']='us-west-1'
def create_spark_session(): 
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .config("fs.s3n.awsAccessKeyId", aws_access_key_id) \
        .config("fs.s3n.awsSecretAccessKey" , aws_secret_access_key) \
        .getOrCreate() 
    return spark
spark = create_spark_session() 
event_data = "s3a://udacity-dsnd/sparkify/sparkify_event_data.json"
print("reading data")
df = spark.read.json(event_data)

reading data

In [5]:
%run Users/tw2567@columbia.edu/Udacity/churn_prediction/util

In [6]:
dp = DataPreparation(df)

Numbers of rows in the data: 26,259,199
Numbers of rows with N/A user Ids: 0
Numbers of unique customers: 22,278

In [7]:
user_activities = dp.user_activities

In [8]:
display(user_activities.select('days_before_today').groupby().max())

max(days_before_today)
61


Unfortunately we only have customers' activity data for 2 months...

#Definition of "being chruned"
**Churn:** either downgrade from paid level to free level or cancel the service
<br>**Data used to predict:** Data till today
<br>**Customers qualified for prediction:** Customers who are still active today

Ideally we can predict churn activities that will happen for a longer time period (e.g. 14 days after today ~ 30 days after today). But because we only have data for 2 months, we have to cut down the time period a little bit....

We would like to predict the churn activities for a "further" future not for tomorrow because we would want to have some time to act (like running rentention campaigns).

When we train the model, imagine that we collect the customer activity data today. We will use the data in the past 7 days to calculate the "training y" and use the data from day1 till 14 days before. If, in real life, we have data for longer period, we can roll the performance measuring window to get more training data points.

In [11]:
last_status = user_activities\
              .withColumn('rownum', F.row_number().over(Window.partitionBy("userId").orderBy(F.col('time').desc())))\
              .filter('rownum==1')\
              .select('userId',
                      F.when(F.col('page')=='Cancel', 1)\
                       .when(F.col('page')=='Submit Downgrade', 1)\
                       .otherwise(0).alias('last_status_cancelled'))

In [12]:
print(last_status.count())
print(last_status.select('userId').distinct().count())

22277
22277

In [13]:
display(user_activities\
              .withColumn('rownum', F.row_number().over(Window.partitionBy("userId").orderBy(F.col('time').desc())))\
              .filter("rownum!=1 and page='Cancel'"))

userId,registration,registration_ts,level,time,timestamp,time_in_the_day,sessionId,itemInSession,page,artist,singer-song,length,days_since_registration,days_before_today,rownum


In [14]:
churned = last_status.filter("last_status_cancelled = 1")

In [15]:
print("Churn rate: {:.2%}".format(churned.count()/user_activities.select('userId').distinct().count()))

Churn rate: 22.47%

# Prepare training data

In this section we will prepare the data for modeling.

In [17]:
%run Users/tw2567@columbia.edu/Udacity/churn_prediction/util

In [18]:
dp = DataPreparation(df)

Numbers of rows in the data: 26,259,199
Numbers of rows with N/A user Ids: 0
Numbers of unique customers: 22,278

In [19]:
featues = dp.run()

Investigate whether any customer has null static features.

In [21]:
featues.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in ['female','customer_age']]).show()

+------+------------+
female|customer_age|
+------+------------+
 0| 1|
+------+------------+

In [22]:
display(featues.filter("customer_age is Null"))

userId,female,state_AZ,state_SC,state_LA,state_MN,state_NJ,state_DC,state_OR,state_VA,state_RI,state_KY,state_WY,state_NH,state_MI,state_NV,state_WI,state_ID,state_CA,state_CT,state_NE,state_MT,state_NC,state_VT,state_MD,state_DE,state_MO,state_IL,state_ME,state_WA,state_ND,state_MS,state_AL,state_IN,state_OH,state_TN,state_IA,state_NM,state_PA,state_SD,state_NY,state_TX,state_WV,state_GA,state_MA,state_KS,state_FL,state_CO,state_AK,state_AR,state_OK,state_UT,state_HI,customer_age,agent_Firefox,agent_Mozilla,agent_Safari,agent_Gecko,agent_GSA,agent_Version,agent_Chromium,agent_Ubuntu,agent_AppleWebKit,agent_Mobile,agent_Chrome,sessions,items,active_days,active_time,sessions_per_day,time_per_day,avg_items_per_session,time_per_session,perc_1_6,perc_6_12,perc_12_18,perc_18_1,unique_songs,unique_artists,max_song_perc,numbers_thup,numbers_thdn,numbers_addlist,numbers_addfrd,numbers_error,perc_thup,perc_thdn,perc_addlist,perc_addfrd,perc_popular_songs,last_status_paid,paid_days,perc_paid_days
1261737,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [23]:
display(df.filter("userId = 1261737").limit(10))

artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
null,Logged Out,null,null,87,null,null,paid,null,GET,Home,null,8615,null,200,1538352008000,null,1261737
null,Logged Out,null,null,0,null,null,free,null,PUT,Login,null,7433,null,307,1538352041000,null,1261737
null,Logged Out,null,null,4,null,null,free,null,GET,Home,null,25003,null,200,1538352182000,null,1261737
null,Logged Out,null,null,2,null,null,free,null,GET,Home,null,9930,null,200,1538352254000,null,1261737
null,Logged Out,null,null,3,null,null,free,null,PUT,Login,null,9930,null,307,1538352255000,null,1261737
null,Logged Out,null,null,0,null,null,free,null,PUT,Login,null,23471,null,307,1538352259000,null,1261737
null,Logged Out,null,null,44,null,null,free,null,GET,Home,null,6317,null,200,1538352278000,null,1261737
null,Logged Out,null,null,43,null,null,free,null,GET,Home,null,22951,null,200,1538352361000,null,1261737
null,Logged Out,null,null,44,null,null,free,null,GET,Home,null,22951,null,200,1538352365000,null,1261737
null,Logged Out,null,null,95,null,null,paid,null,GET,Home,null,6071,null,200,1538352404000,null,1261737


Actually the customer with the ID 1261737 seems wierd.

In [25]:
display(featues.limit(20))

userId,female,state_AZ,state_SC,state_LA,state_MN,state_NJ,state_DC,state_OR,state_VA,state_RI,state_KY,state_WY,state_NH,state_MI,state_NV,state_WI,state_ID,state_CA,state_CT,state_NE,state_MT,state_NC,state_VT,state_MD,state_DE,state_MO,state_IL,state_ME,state_WA,state_ND,state_MS,state_AL,state_IN,state_OH,state_TN,state_IA,state_NM,state_PA,state_SD,state_NY,state_TX,state_WV,state_GA,state_MA,state_KS,state_FL,state_CO,state_AK,state_AR,state_OK,state_UT,state_HI,customer_age,agent_Firefox,agent_Mozilla,agent_Safari,agent_Gecko,agent_GSA,agent_Version,agent_Chromium,agent_Ubuntu,agent_AppleWebKit,agent_Mobile,agent_Chrome,sessions,items,active_days,active_time,sessions_per_day,time_per_day,avg_items_per_session,time_per_session,perc_1_6,perc_6_12,perc_12_18,perc_18_1,unique_songs,unique_artists,max_song_perc,numbers_thup,numbers_thdn,numbers_addlist,numbers_addfrd,numbers_error,perc_thup,perc_thdn,perc_addlist,perc_addfrd,perc_popular_songs,last_status_paid,paid_days,perc_paid_days
1000280,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,95,1,1,0,1,0,0,0,0,0,0,0,22,1316,21,257231000,1.1428571428571428,1.2226666666666666E7,59.81818181818182,1.1692318181818182E7,0.27337375877553605,0.07540636265890835,0.356836221617861,0.29438365694769464,948,767,0.00806426022179331,53,33,25,14,3,0.04027355623100304,0.02507598784194529,0.018996960486322188,0.010638297872340425,0.16046966731898238,0,1,0.045454545454545456
1002185,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,68,0,1,1,0,0,0,0,0,1,0,1,17,2080,21,443434000,1.0952380952380953,2.103909523809524E7,122.3529411764706,2.608435294117647E7,0.23289086863770825,0.22848455268904425,0.2889402157101318,0.24968436296311572,1582,1205,0.007399086752101751,92,14,49,25,2,0.04423076923076923,0.006730769230769231,0.023557692307692307,0.01201923076923077,0.15748031496062992,1,21,1.0
1017805,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,90,0,1,1,0,0,1,0,0,1,0,0,3,320,4,63410000,1.0,1.578475E7,106.66666666666667,2.1136666666666668E7,0.25327994682450505,0.0,0.5325304255622201,0.21418962761327487,245,223,0.011018552257797272,7,4,5,13,0,0.021875,0.0125,0.015625,0.040625,0.14,1,4,1.0
1030587,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,133,0,1,1,0,0,0,0,0,1,0,1,11,1752,13,362380000,1.1538461538461537,2.7668769230769232E7,159.27272727272728,3.2943636363636363E7,0.35346876229185786,0.2864494558804248,0.15136808378130326,0.20871369804641404,1345,1071,0.00900083941476212,66,16,46,23,1,0.03767123287671233,0.0091324200913242,0.026255707762557076,0.013127853881278538,0.14130434782608695,1,12,0.8571428571428571
1033297,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,120,1,1,0,1,0,0,0,0,0,0,0,5,299,6,59105000,1.0,9821000.0,59.8,1.1821E7,0.17523205023205024,0.07725907725907726,0.3840601965601966,0.36344867594867597,234,215,0.016470209486091937,10,3,7,4,0,0.033444816053511704,0.010033444816053512,0.023411371237458192,0.013377926421404682,0.1652542372881356,1,2,0.2857142857142857
1057724,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,98,0,1,1,0,0,0,0,0,1,0,1,40,4669,41,953791000,1.2195121951219512,2.3195975609756097E7,116.725,2.3844775E7,0.17366028426461722,0.2748881677045873,0.2990831371675157,0.2523684108632797,3190,2157,0.009595024447421621,200,29,135,76,1,0.04283572499464553,0.006211180124223602,0.028914114371385736,0.016277575497965303,0.16246425786327007,1,24,0.5714285714285714
1059049,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,143,0,1,1,0,0,0,0,0,1,0,1,5,662,5,140598000,1.2,2.80482E7,132.4,2.81196E7,0.503600432737259,0.16671643471492698,0.20805654955673802,0.12162658299107602,535,454,0.01852620233627137,29,6,16,1

In [26]:
featues.filter("userId != 1261737").fillna(0).join(last_status, on="userId")\
.write.parquet("s3://tianyi-wang-data-science-projects/churn-prediction-2020/features1", mode='overwrite')